In [32]:
import cv2
import time
import mediapipe as mp
import numpy as np
import random

# Inicializar Mediapipe para detección de manos y rostro
mp_hands = mp.solutions.hands
mp_face_mesh = mp.solutions.face_mesh
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2,
                       min_detection_confidence=0.5, min_tracking_confidence=0.5)
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1,
                                  min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Cargar la imagen de la manzana
apple_img = cv2.imread('apple.png', cv2.IMREAD_UNCHANGED)
apple_img = cv2.resize(apple_img, (50, 50))  # Ajustar el tamaño de la manzana

# Si no se puede cargar la imagen, usar un círculo como manzana
if apple_img is None or apple_img.shape[2] != 4:
    apple_img = np.zeros((50, 50, 4), dtype=np.uint8)
    cv2.circle(apple_img, (25, 25), 25, (0, 0, 255, 255), -1)

# Cargar la imagen de la piedra
stone_img = cv2.imread('stone.png', cv2.IMREAD_UNCHANGED)
stone_img = cv2.resize(stone_img, (50, 50))  # Ajustar el tamaño de la piedra

# Si no se puede cargar la imagen, usar un cuadrado gris como piedra
if stone_img is None or stone_img.shape[2] != 4:
    stone_img = np.zeros((50, 50, 4), dtype=np.uint8)
    cv2.rectangle(stone_img, (0, 0), (50, 50), (128, 128, 128, 255), -1)

# Cargar la imagen de la vida (corazón)
life_img = cv2.imread('heart.png', cv2.IMREAD_UNCHANGED)
life_img = cv2.resize(life_img, (30, 30))  # Ajustar el tamaño del corazón

# Si no se puede cargar la imagen, usar un círculo rojo como vida
if life_img is None or life_img.shape[2] != 4:
    life_img = np.zeros((30, 30, 4), dtype=np.uint8)
    cv2.circle(life_img, (15, 15), 15, (0, 0, 255, 255), -1)

# Cargar la imagen de la boca gigante
big_mouth_img = cv2.imread('comemanzanas.png', cv2.IMREAD_UNCHANGED)
big_mouth_original_size = (150, 150)  # Tamaño por defecto si no se puede cargar
if big_mouth_img is None or big_mouth_img.shape[2] != 4:
    big_mouth_img = np.zeros((big_mouth_original_size[1], big_mouth_original_size[0], 4), dtype=np.uint8)
    cv2.circle(big_mouth_img, (big_mouth_original_size[0]//2, big_mouth_original_size[1]//2),
               min(big_mouth_original_size)//2, (0, 0, 255, 255), -1)  # Círculo rojo por defecto

# Parámetros del juego
lives = 3
apples = []
stones = []
apple_spawn_interval = 5  # Intervalo inicial para crear manzanas (en segundos)
stone_spawn_interval = 8  # Intervalo inicial para crear piedras
last_apple_spawn_time = 0
last_stone_spawn_time = 0
game_started = False
game_over = False
start_time = time.time()
difficulty_increase_interval = 15  # Cada 15 segundos aumenta la dificultad
apple_fall_speed = 2  # Velocidad inicial de caída de las manzanas
stone_fall_speed = 2
max_apples_on_screen = 1  # Número máximo inicial de manzanas en pantalla
max_stones_on_screen = 0  # Las piedras comienzan a aparecer después

# Captura de video
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        print('Error al abrir la cámara')
        exit(0)

# Función para superponer una imagen con transparencia
def overlay_image_alpha(img, img_overlay, pos):
    x, y = int(pos[0]), int(pos[1])
    overlay_h, overlay_w = img_overlay.shape[:2]

    # Ajustar si x o y son negativos
    if x < 0:
        img_overlay = img_overlay[:, -x:]
        overlay_w = img_overlay.shape[1]
        x = 0
    if y < 0:
        img_overlay = img_overlay[-y:, :]
        overlay_h = img_overlay.shape[0]
        y = 0

    # Asegurarse de que la imagen de superposición esté dentro de los límites
    if x + overlay_w > img.shape[1]:
        overlay_w = img.shape[1] - x
        img_overlay = img_overlay[:, :overlay_w]
    if y + overlay_h > img.shape[0]:
        overlay_h = img_overlay.shape[0] - y
        img_overlay = img_overlay[:overlay_h, :]

    if overlay_w <= 0 or overlay_h <= 0:
        return img

    # Extraer canales
    alpha_overlay = img_overlay[:, :, 3] / 255.0
    alpha_background = 1.0 - alpha_overlay

    # Superponer imagen
    for c in range(0, 3):
        img[y:y+overlay_h, x:x+overlay_w, c] = (alpha_overlay * img_overlay[:, :, c] +
                                                alpha_background * img[y:y+overlay_h, x:x+overlay_w, c])
    return img

# Clase para los objetos (manzanas y piedras)
class GameObject:
    def __init__(self, x, y, fall_speed, obj_type='apple'):
        self.x = x
        self.y = y
        self.state = 'falling'  # 'falling', 'to_mouth'
        self.fall_speed = fall_speed
        self.path = []
        self.current_path_index = 0
        self.type = obj_type  # 'apple' o 'stone'

    def update_position(self):
        if self.state == 'falling':
            self.y += self.fall_speed
        elif self.state == 'to_mouth':
            if self.current_path_index < len(self.path):
                self.x, self.y = self.path[self.current_path_index]
                self.current_path_index += 1
            else:
                # Si ha terminado el camino hacia la boca
                return 'reached_mouth'
        return 'continue'

    def generate_parabola(self, mouth_x, mouth_y):
        # Generar puntos para una parábola desde la posición actual hasta la boca
        num_points = 30  # A mayor número, más suave la parábola
        x1, y1 = self.x, self.y
        x2, y2 = mouth_x, mouth_y

        # Evitar división por cero
        if x2 - x1 == 0:
            x2 += 1

        # Generar puntos de una parábola simple
        self.path = []
        for t in np.linspace(0, 1, num_points):
            x = (1 - t) * x1 + t * x2
            y = (1 - t) * y1 + t * y2 - 100 * t * (1 - t)  # Ajusta el valor 100 para cambiar la curvatura
            self.path.append((int(x), int(y)))

# Estado de la boca
mouth_open = False

# Variable para almacenar la caja delimitadora del rostro
current_face_bbox = None

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Flip horizontal para efecto espejo
    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Procesar detección de manos
    hand_results = hands.process(rgb_frame)

    # Procesar detección de rostro
    face_results = face_mesh.process(rgb_frame)

    # Obtener dimensiones del frame
    h, w, _ = frame.shape

    # Reiniciar la caja delimitadora del rostro
    current_face_bbox = None

    # Variables para almacenar posiciones de manos y boca
    hand_positions = []
    mouth_coords = {}

    # Detectar manos y extraer posiciones
    if hand_results.multi_hand_landmarks:
        for hand_landmarks in hand_results.multi_hand_landmarks:
            # Obtener coordenadas de los puntos de la mano
            for lm in hand_landmarks.landmark:
                x = int(lm.x * w)
                y = int(lm.y * h)
                hand_positions.append((x, y))

            # Dibujar puntos de la mano
            mp.solutions.drawing_utils.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    # Detectar rostro y extraer posiciones de los labios
    if face_results.multi_face_landmarks:
        for face_landmarks in face_results.multi_face_landmarks:
            # Obtener coordenadas de los labios
            lip_top = face_landmarks.landmark[13]
            lip_bottom = face_landmarks.landmark[14]
            lip_left = face_landmarks.landmark[78]
            lip_right = face_landmarks.landmark[308]

            # Convertir a píxeles
            top_lip_y = int(lip_top.y * h)
            bottom_lip_y = int(lip_bottom.y * h)
            left_lip_x = int(lip_left.x * w)
            right_lip_x = int(lip_right.x * w)

            # Calcular apertura de la boca
            mouth_opening = abs(top_lip_y - bottom_lip_y)

            # Umbral para considerar boca abierta
            if mouth_opening > 7:
                mouth_open = True
            else:
                mouth_open = False

            # Coordenadas de la boca
            mouth_coords = {'x': int((left_lip_x + right_lip_x) / 2),
                            'y': int((top_lip_y + bottom_lip_y) / 2)}

            # Dibujar puntos de la boca
            cv2.circle(frame, (mouth_coords['x'], mouth_coords['y']), 2, (0, 255, 0), -1)

            # Calcular caja delimitadora del rostro
            x_coords = [int(lm.x * w) for lm in face_landmarks.landmark]
            y_coords = [int(lm.y * h) for lm in face_landmarks.landmark]
            min_x, max_x = min(x_coords), max(x_coords)
            min_y, max_y = min(y_coords), max(y_coords)
            face_width = max_x - min_x
            face_height = max_y - min_y

            # Guardar la caja delimitadora actual
            current_face_bbox = (min_x, min_y, max_x, max_y)

            # Ajustar el tamaño de la boca gigante según el rostro
            # Puedes ajustar estos factores según la apariencia deseada
            big_mouth_width = int(face_width * 1.6)
            big_mouth_height = int(face_height * 1.6)
            resized_big_mouth = cv2.resize(big_mouth_img, (big_mouth_width, big_mouth_height))

            # Calcular posición para centrar la boca gigante
            mouth_center_x = int((min_x + max_x) / 2)
            mouth_center_y = int((min_y + max_y) / 2)
            top_left_x = mouth_center_x - big_mouth_width // 2
            top_left_y = mouth_center_y - big_mouth_height // 2

            # Superponer la boca gigante si la boca está abierta
            if mouth_open:
                frame = overlay_image_alpha(frame, resized_big_mouth, (top_left_x, top_left_y))

    # Estado inicial: esperando gesto de manos juntas
    if not game_started and not game_over:
        if hand_results.multi_hand_landmarks and len(hand_results.multi_hand_landmarks) == 2:
            # Calcular distancia entre las dos primeras posiciones detectadas
            x1, y1 = hand_positions[0]
            x2, y2 = hand_positions[1]
            palm_distance = np.hypot(x2 - x1, y2 - y1)

            # Si las palmas están cerca (umbral ajustable)
            if palm_distance < 27:
                game_started = True
                start_time = time.time()
        else:
            cv2.putText(frame, "Junta tus manos para comenzar", (50, 50), cv2.FONT_HERSHEY_SIMPLEX,
                        1, (255, 255, 255), 2)

    # Si el juego ha comenzado
    if game_started and not game_over:
        current_time = time.time()
        elapsed_time = int(current_time - start_time)

        # Mostrar temporizador
        cv2.putText(frame, f"Tiempo: {elapsed_time}s", (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                    1, (255, 255, 255), 2)

        # Aumentar dificultad cada cierto tiempo
        if elapsed_time > 0 and elapsed_time % difficulty_increase_interval == 0:
            apple_fall_speed += 0.5  # Aumentar velocidad de caída de manzanas
            stone_fall_speed += 0.5  # Aumentar velocidad de caída de piedras
            apple_spawn_interval = max(0.5, apple_spawn_interval - 0.5)  # Disminuir intervalo de aparición
            max_apples_on_screen += 1  # Aumentar número máximo de manzanas
            if max_stones_on_screen < 3:
                max_stones_on_screen += 1  # Aumentar número máximo de piedras hasta 3
            difficulty_increase_interval += 15  # Aumentar el tiempo para la siguiente dificultad

        # Generar nuevas manzanas si es necesario
        if len(apples) < max_apples_on_screen and current_time - last_apple_spawn_time > apple_spawn_interval:
            apple_x = random.randint(25, w - 25)
            apples.append(GameObject(apple_x, -25, apple_fall_speed, obj_type='apple'))
            last_apple_spawn_time = current_time

        # Generar nuevas piedras si es necesario
        if elapsed_time > 20:  # Las piedras empiezan a aparecer después de 20 segundos
            if len(stones) < max_stones_on_screen and current_time - last_stone_spawn_time > stone_spawn_interval:
                stone_x = random.randint(25, w - 25)
                stones.append(GameObject(stone_x, -25, stone_fall_speed, obj_type='stone'))
                last_stone_spawn_time = current_time

        # Actualizar y dibujar manzanas
        for apple in apples[:]:
            status = apple.update_position()

            # Dibujar manzana
            frame = overlay_image_alpha(frame, apple_img, (apple.x - 25, apple.y - 25))

            # Si la manzana está cayendo
            if apple.state == 'falling':
                # Comprobar si la manzana toca el suelo
                if apple.y >= h - 25:
                    apples.remove(apple)
                    lives -= 1
                    if lives == 0:
                        game_over = True
                        total_time = elapsed_time
                        break
                    continue

                # Comprobar colisión con manos
                for hx, hy in hand_positions:
                    if abs(apple.x - hx) < 30 and abs(apple.y - hy) < 30:
                        # Generar trayectoria hacia la boca
                        if mouth_open and current_face_bbox is not None:
                            # Usar el centro de la caja delimitadora del rostro
                            face_min_x, face_min_y, face_max_x, face_max_y = current_face_bbox
                            mouth_x = int((face_min_x + face_max_x) / 2)
                            mouth_y = int((face_min_y + face_max_y) / 2)
                        else:
                            mouth_x = mouth_coords.get('x', w // 2)
                            mouth_y = mouth_coords.get('y', h // 2)
                        apple.generate_parabola(mouth_x, mouth_y)
                        apple.state = 'to_mouth'
                        break  # No es necesario comprobar más manos

            # Si la manzana va hacia la boca
            elif apple.state == 'to_mouth':
                # Comprobar si ha salido de la pantalla
                if apple.x < 0 or apple.x > w or apple.y < 0 or apple.y > h:
                    apples.remove(apple)
                    lives -= 1
                    if lives == 0:
                        game_over = True
                        total_time = elapsed_time
                        break
                    continue

                if status == 'reached_mouth':
                    if not mouth_open:
                        lives -= 1  # Restar una vida si la boca está cerrada
                        if lives == 0:
                            game_over = True
                            total_time = elapsed_time
                    apples.remove(apple)
                    continue

            # Comprobar si la manzana está frente al rostro y la boca está abierta
            if mouth_open and current_face_bbox is not None:
                face_min_x, face_min_y, face_max_x, face_max_y = current_face_bbox
                if face_min_x <= apple.x <= face_max_x and face_min_y <= apple.y <= face_max_y:
                    apples.remove(apple)
                    continue  # La manzana ha sido comida

        # Actualizar y dibujar piedras
        for stone in stones[:]:
            status = stone.update_position()

            # Dibujar piedra
            frame = overlay_image_alpha(frame, stone_img, (stone.x - 25, stone.y - 25))

            # Si la piedra está cayendo
            if stone.state == 'falling':
                # Comprobar si la piedra toca el suelo
                if stone.y >= h - 25:
                    stones.remove(stone)
                    continue  # Las piedras que caen no afectan las vidas

                # Comprobar colisión con manos
                for hx, hy in hand_positions:
                    if abs(stone.x - hx) < 30 and abs(stone.y - hy) < 30:
                        # Generar trayectoria hacia la boca
                        if mouth_open and current_face_bbox is not None:
                            # Usar el centro de la caja delimitadora del rostro
                            face_min_x, face_min_y, face_max_x, face_max_y = current_face_bbox
                            mouth_x = int((face_min_x + face_max_x) / 2)
                            mouth_y = int((face_min_y + face_max_y) / 2)
                        else:
                            mouth_x = mouth_coords.get('x', w // 2)
                            mouth_y = mouth_coords.get('y', h // 2)
                        stone.generate_parabola(mouth_x, mouth_y)
                        stone.state = 'to_mouth'
                        break  # No es necesario comprobar más manos

            # Si la piedra va hacia la boca
            elif stone.state == 'to_mouth':
                # Comprobar si ha salido de la pantalla
                if stone.x < 0 or stone.x > w or stone.y < 0 or stone.y > h:
                    stones.remove(stone)
                    continue  # Las piedras que salen de pantalla no afectan las vidas

                if status == 'reached_mouth':
                    stones.remove(stone)
                    continue

            # Comprobar si la piedra está frente al rostro y la boca está abierta
            if mouth_open and current_face_bbox is not None:
                face_min_x, face_min_y, face_max_x, face_max_y = current_face_bbox
                if face_min_x <= stone.x <= face_max_x and face_min_y <= stone.y <= face_max_y:
                    stones.remove(stone)
                    lives -= 1
                    if lives == 0:
                        game_over = True
                        total_time = elapsed_time
                        break

        # Dibujar vidas en la esquina superior derecha
        for i in range(lives):
            frame = overlay_image_alpha(frame, life_img, (w - (i + 1) * 40, 10))

    if game_over:
        cv2.putText(frame, "Juego Terminado", (w // 2 - 200, h // 2 - 30), cv2.FONT_HERSHEY_SIMPLEX,
                    1.5, (0, 0, 255), 3)
        cv2.putText(frame, f"Tiempo total: {total_time}s", (w // 2 - 150, h // 2 + 10), cv2.FONT_HERSHEY_SIMPLEX,
                    1, (255, 255, 255), 2)
        cv2.putText(frame, "Presiona 'R' para reiniciar", (w // 2 - 200, h // 2 + 50), cv2.FONT_HERSHEY_SIMPLEX,
                    1, (255, 255, 255), 2)

    # Mostrar la imagen
    cv2.imshow('Minijuego: Come la Manzana', frame)

    # Esperar por tecla de salida
    tec = cv2.waitKey(1)
    if tec & 0xFF == 27:  # Esc para salir
        break
    elif tec & 0xFF == ord('r') and game_over:
        # Reiniciar juego
        lives = 3
        apples = []
        stones = []
        apple_spawn_interval = 5
        stone_spawn_interval = 8
        last_apple_spawn_time = 0
        last_stone_spawn_time = 0
        game_started = False
        game_over = False
        apple_fall_speed = 2
        stone_fall_speed = 2
        max_apples_on_screen = 1
        max_stones_on_screen = 0
        start_time = time.time()
        difficulty_increase_interval = 15

# Cerrar cámara y ventanas
cap.release()
cv2.destroyAllWindows()

